# Examining the Effects of DP on Exponential Smoothing Forecast Accuracy

***

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import *

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

***

## SES

In [3]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [4]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [5]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [6]:
results_dict_ses

{'h=1, epsilon = 1': {'Mean Accuracies': array([2.18, 2.18]),
  'Protected Mean Accuracies:': array([127.23, 127.23]),
  '% Change Mean accuracy:': array([-5730.92, -5730.92]),
  '% Change Median accuracy:': array([-12554.45, -12554.45]),
  '% Forecasted Points adjusted downward:': 3.66,
  '% Forecasted Points adjusted upward:': 96.34,
  '% Series with improved accuracy:': array([0., 0.]),
  '% Series with reduced accuracy:': array([100., 100.]),
  'Original Mean Absolute Error Upward Adjusted:': 2.25,
  'Original Mean Absolute Error Downward Adjusted:': 0.33,
  'Protected Mean Absolute Error Upward Adjusted:': 131.79000000000002,
  'Protected Mean Absolute Error Downward Adjusted:': 7.21},
 'h=20, epsilon = 1': {'Mean Accuracies': array([4.4 , 5.37]),
  'Protected Mean Accuracies:': array([120.5, 120.6]),
  '% Change Mean accuracy:': array([-2637.98, -2145.26]),
  '% Change Median accuracy:': array([-3627.6 , -3066.94]),
  '% Forecasted Points adjusted downward:': 6.98,
  '% Forecaste

***
***

In [7]:
original_forecasts = fcasts_ses['h=20, epsilon = 20']
protected_forecasts = fcasts_protected_ses['h=20, epsilon = 20']
test = tests['h=20, epsilon = 20']

In [8]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [13]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [15]:
np.mean(adjusted_down[improved])

0.5042801556420233

In [16]:
np.mean(adjusted_up[improved])

0.49571984435797667

***
***

## DES

In [7]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [8]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [9]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [10]:
results_dict_des

{'h=1, epsilon = 1': {'Mean Accuracies': array([2.1, 2.1]),
  'Protected Mean Accuracies:': array([127.25, 127.25]),
  '% Change Mean accuracy:': array([-5945.49, -5945.49]),
  '% Change Median accuracy:': array([-12265.06, -12265.06]),
  '% Forecasted Points adjusted downward:': 2.44,
  '% Forecasted Points adjusted upward:': 97.56,
  '% Series with improved accuracy:': array([0.61, 0.61]),
  '% Series with reduced accuracy:': array([99.39, 99.39]),
  'Original Mean Absolute Error Upward Adjusted:': 2.15,
  'Original Mean Absolute Error Downward Adjusted:': 0.41000000000000003,
  'Protected Mean Absolute Error Upward Adjusted:': 130.29,
  'Protected Mean Absolute Error Downward Adjusted:': 5.6899999999999995},
 'h=20, epsilon = 1': {'Mean Accuracies': array([4.24, 5.18]),
  'Protected Mean Accuracies:': array([123.6 , 123.72]),
  '% Change Mean accuracy:': array([-2818. , -2288.5]),
  '% Change Median accuracy:': array([-3664.37, -2952.29]),
  '% Forecasted Points adjusted downward:':

***
***

## TES

In [11]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [12]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [13]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [14]:
results_dict_tes

{'h=1, epsilon = 1': {'Mean Accuracies': array([2.44, 2.44]),
  'Protected Mean Accuracies:': array([123.93, 123.93]),
  '% Change Mean accuracy:': array([-4975.09, -4975.09]),
  '% Change Median accuracy:': array([-9975.42, -9975.42]),
  '% Forecasted Points adjusted downward:': 1.83,
  '% Forecasted Points adjusted upward:': 98.17,
  '% Series with improved accuracy:': array([0., 0.]),
  '% Series with reduced accuracy:': array([100., 100.]),
  'Original Mean Absolute Error Upward Adjusted:': 2.45,
  'Original Mean Absolute Error Downward Adjusted:': 1.87,
  'Protected Mean Absolute Error Upward Adjusted:': 126.01,
  'Protected Mean Absolute Error Downward Adjusted:': 12.06},
 'h=20, epsilon = 1': {'Mean Accuracies': array([4.75, 5.74]),
  'Protected Mean Accuracies:': array([124.86, 125.54]),
  '% Change Mean accuracy:': array([-2530.16, -2085.66]),
  '% Change Median accuracy:': array([-3550.65, -2862.79]),
  '% Forecasted Points adjusted downward:': 3.1399999999999997,
  '% Foreca